In [1]:
# Run all matra thiche pugcha

In [2]:
import os
import re

# Path to the main directory containing checkpoints
base_dir = "results"

# List all folders in the directory
folders = os.listdir(base_dir)

# Filter for checkpoint folders and extract their numbers
checkpoints = [
    (folder, int(re.search(r"checkpoint-(\d+)", folder).group(1)))
    for folder in folders
    if re.match(r"checkpoint-\d+", folder)
]

# Get the folder with the highest checkpoint number
max_checkpoint = max(checkpoints, key=lambda x: x[1])[0]

# Full path to the checkpoint directory
checkpoint_path = "./results/" + max_checkpoint

# Output
print("Max checkpoint path:", checkpoint_path)
# checkpoint_path = "./results/checkpoint-41000"


Max checkpoint path: ./results/checkpoint-80000


In [3]:
# !pip install transformers==4.41.1 datasets==2.18.0 evaluate==0.4.1 torch==2.3.0 accelerate==0.31.0

In [4]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [5]:

from datasets import load_dataset

# Load arXiv summarization dataset
dataset = load_dataset("ccdv/arxiv-summarization")
# dataset = dataset.select(range(1000))   # remove later
sample = dataset["train"][0]
print("Article:", sample["article"][:500])
print("Summary:", sample["abstract"])


c:\Users\PC\anaconda3\envs\longt5\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Article: additive models @xcite provide an important family of models for semiparametric regression or classification . some reasons for the success of additive models are their increased flexibility when compared to linear or generalized linear models and their increased interpretability when compared to fully nonparametric models . 
 it is well - known that good estimators in additive models are in general less prone to the curse of high dimensionality than good estimators in fully nonparametric models
Summary: additive models play an important role in semiparametric statistics . 
 this paper gives learning rates for regularized kernel based methods for additive models . 
 these learning rates compare favourably in particular in high dimensions to recent results on optimal learning rates for purely nonparametric regularized kernel based quantile regression using the gaussian radial basis function kernel , provided the assumption of an additive model is valid . 
 additionally , a conc

In [6]:

# from transformers import LongT5ForConditionalGeneration, AutoTokenizer

# model_name = "google/long-t5-tglobal-base"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = LongT5ForConditionalGeneration.from_pretrained(model_name)
# model.config.use_cache = False
# model.gradient_checkpointing_enable() 


In [7]:
from transformers import LongT5ForConditionalGeneration, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = LongT5ForConditionalGeneration.from_pretrained(
    checkpoint_path,
    local_files_only=True,
    ignore_mismatched_sizes=True  # Optional: avoid shape mismatch crashes
)

model.config.use_cache = False
model.gradient_checkpointing_enable()


In [8]:
def preprocess_function(batch):
    inputs = ["summarize: " + doc for doc in batch["article"]]
    model_inputs = tokenizer(inputs, max_length=4096, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["abstract"], max_length=256, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [9]:
# Assuming your dataset has a 'train' split
# tokenized_dataset = dataset["train"].map(
#     preprocess_function,
#     batched=True,
#     remove_columns=["article", "abstract"]
# )

# tokenized_dataset.save_to_disk("tokenized_dataset")
from datasets import load_from_disk
tokenized_dataset = load_from_disk("tokenized_dataset")

# Then perform train-test split on the tokenized dataset
split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]
# 53 minutes

In [10]:
from transformers import DataCollatorForSeq2Seq
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)


def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    accumulated_fmeasures = {
        'rouge1': 0.0,
        'rouge2': 0.0,
        'rougeL': 0.0, 
    }
    num_samples = 0

    for pred_text, ref_text in zip(decoded_preds, decoded_labels):
        scores = scorer.score(ref_text, pred_text)

        for key in accumulated_fmeasures:
            if key in scores:
                accumulated_fmeasures[key] += scores[key].fmeasure
        num_samples += 1

    average_metrics = {}
    if num_samples > 0:
        for key, total_fmeasure in accumulated_fmeasures.items():
            average_metrics[key] = (total_fmeasure / num_samples) * 100

    return average_metrics

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, label_pad_token_id=-100)

In [11]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0)) 
print(torch.backends.cudnn.version()) 

2.5.1+cu121
12.1
True
NVIDIA GeForce RTX 3060
90100


In [12]:
import torch

def print_gpu_memory():
    if torch.cuda.is_available():
        print("Allocated:", torch.cuda.memory_allocated() / 1024**3, "GB")
        print("Cached:   ", torch.cuda.memory_reserved() / 1024**3, "GB")
    else:
        print("CUDA not available.")

print_gpu_memory()

Allocated: 0.0 GB
Cached:    0.0 GB


In [13]:
from transformers import TrainingArguments
import os

# Make sure results directory exists and is writable
os.makedirs("./results", exist_ok=True)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    # eval_steps=5000,                      # Reduce frequency if not needed
    save_strategy="steps",
    save_steps=1000,                      # Keep it if you want frequent saves
    save_total_limit=2,
    load_best_model_at_end=False,         # OPTIONAL: Turn off to simplify
    metric_for_best_model="rougeL",
    greater_is_better=True,
    learning_rate=1e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    eval_accumulation_steps=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=200,
    # fp16=torch.cuda.is_available(),
    fp16=False,
    gradient_accumulation_steps=2,
    max_grad_norm=1.0,
    disable_tqdm=False,                   # Enable tqdm to check live logs
    report_to=[],                         # Avoid WandB etc.
    save_safetensors=True,                # ✅ Save in safer format
)


c:\Users\PC\anaconda3\envs\longt5\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
# import torch
# torch.cuda.empty_cache()
# from transformers import TrainingArguments


# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="steps",
#     eval_steps=1000,
#     save_strategy="steps",
#     save_steps=1000, 
#     save_total_limit=2,
#     load_best_model_at_end=True,
#     metric_for_best_model="rougeL",
#     greater_is_better=True,
#     learning_rate=3e-5,
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=1,
#     eval_accumulation_steps=4, 
#     num_train_epochs=1,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     logging_steps=200,

#     fp16=torch.cuda.is_available(),
#     gradient_accumulation_steps=2,
#     max_grad_norm=1.0,
# )




In [15]:
from transformers import TrainerCallback
import torch

class ClipNanGradientsCallback(TrainerCallback):    
    def on_step_end(self, args, state, control, model=None, **kwargs):
        if model is not None:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)


In [16]:

from transformers import Trainer, EarlyStoppingCallback

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=None,
    callbacks=[ClipNanGradientsCallback()],
)

c:\Users\PC\anaconda3\envs\longt5\lib\site-packages\torch\nn\modules\module.py:1326: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


In [17]:
# trainer.train()

# trainer.train(resume_from_checkpoint=True)
# checkpoint_path = "./results/checkpoint-7000"
trainer.train(resume_from_checkpoint=checkpoint_path)



There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].
c:\Users\PC\anaconda3\envs\longt5\lib\site-packages\transformers\trainer.py:3017: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open

{'loss': 1.6468, 'grad_norm': 1.8130735158920288, 'learning_rate': 1.222117636757656e-06, 'epoch': 0.88}


 88%|████████▊ | 80400/91366 [18:27<8:32:18,  2.80s/it]

{'loss': 1.5932, 'grad_norm': 1.369865894317627, 'learning_rate': 1.2002276558019397e-06, 'epoch': 0.88}


 88%|████████▊ | 80600/91366 [27:38<8:02:15,  2.69s/it]

...

c:\Users\PC\anaconda3\envs\longt5\lib\site-packages\transformers\modeling_utils.py:1006: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
100%|█████████▉| 91200/91366 [9:00:36<07:58,  2.88s/it]  

{'loss': 1.6016, 'grad_norm': 1.4305497407913208, 'learning_rate': 1.8168684193244754e-08, 'epoch': 1.0}


100%|██████████| 91366/91366 [9:08:35<00:00,  2.78it/s]

{'train_runtime': 32915.9273, 'train_samples_per_second': 5.552, 'train_steps_per_second': 2.776, 'train_loss': 0.2038577084595285, 'epoch': 1.0}


TrainOutput(global_step=91366, training_loss=0.2038577084595285, metrics={'train_runtime': 32915.9273, 'train_samples_per_second': 5.552, 'train_steps_per_second': 2.776, 'total_flos': 1.001058031632384e+18, 'train_loss': 0.2038577084595285, 'epoch': 0.9999945275347091})

In [18]:

trainer.save_model("./longt5_best_model")
tokenizer.save_pretrained("./longt5_best_model")


('./longt5_best_model\\tokenizer_config.json',
 './longt5_best_model\\special_tokens_map.json',
 './longt5_best_model\\tokenizer.json')

In [ ]:
evaluate

In [3]:
from transformers import LongT5ForConditionalGeneration, AutoTokenizer
from datasets import load_dataset
import torch

# Load dataset
dataset = load_dataset("ccdv/arxiv-summarization")

# Load fine-tuned model and tokenizer
model = LongT5ForConditionalGeneration.from_pretrained("./longt5_best_model")
tokenizer = AutoTokenizer.from_pretrained("./longt5_best_model")

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare input
text = dataset["test"][0]["article"]
input_text = "summarize: " + text
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    max_length=4096,
    truncation=True
)

# Move input to the same device as model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate summary
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=256,
    min_length=30,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)

# Decode summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("\nGenerated Summary:\n", summary)



c:\Users\PC\anaconda3\envs\longt5\lib\site-packages\transformers\modeling_utils.py:1006: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(



Generated Summary:
 in this paper the problem of the existence of the periodicity of about 155 days during the maximum activity period for sunspot data from 1923 - 1933 ( cycle 16 ) is considered. the daily sunspot areas, the mean sunspot areas per carrington rotation, the monthly sunspot numbers and their fluctuations, which are obtained after removing the 11-year cycle are analysed. a new method of the diagnosis of an echo - effect in the power spectrum is presented. numerical results of the new method are presented.


In [9]:
from transformers import LongT5ForConditionalGeneration, AutoTokenizer
from datasets import load_dataset
from rouge_score import rouge_scorer
import torch
from tqdm import tqdm

# Load dataset
dataset = load_dataset("ccdv/arxiv-summarization", split="test")

# Load model and tokenizer
model = LongT5ForConditionalGeneration.from_pretrained("./longt5_best_model")
tokenizer = AutoTokenizer.from_pretrained("./longt5_best_model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Evaluation loop
n_samples = 100  # You can increase to 100 or full len(dataset) for full evaluation
scores = {"rouge1": [], "rouge2": [], "rougeL": []}

for i in tqdm(range(n_samples), desc="Evaluating"):
    article = dataset[i]["article"]
    reference = dataset[i]["abstract"]

    input_text = "summarize: " + article
    inputs = tokenizer(input_text, return_tensors="pt", max_length=4096, truncation=True).to(device)

    summary_ids = model.generate(
        inputs["input_ids"],
        max_length=256,
        min_length=30,
        length_penalty=2.0,
        num_beams=4,
        early_stopping=True
    )

    predicted = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Compute ROUGE scores
    score = scorer.score(reference, predicted)
    for key in scores:
        scores[key].append(score[key].fmeasure)

# Average scores
avg_scores = {key: sum(values) / len(values) for key, values in scores.items()}
print("\nAverage ROUGE Scores on Test Set:")
for key, value in avg_scores.items():
    print(f"{key}: {value:.4f}")


Evaluating: 100%|██████████| 100/100 [10:21<00:00,  6.21s/it]


Average ROUGE Scores on Test Set:
rouge1: 0.4047
rouge2: 0.1505
rougeL: 0.2413


In [ ]:
from transformers import LongT5ForConditionalGeneration, AutoTokenizer
from rouge_score import rouge_scorer
import torch


text_to_summarize = """
In this paper the problem of the existence of the periodicity of about 155 days during the maximum activity period 
for sunspot data from 1923 - 1933 (cycle 16) is considered. The daily sunspot areas, the mean sunspot areas per 
Carrington rotation, the monthly sunspot numbers and their fluctuations, which are obtained after removing the 11-year 
cycle are analysed. A new method of the diagnosis of an echo-effect in the power spectrum is presented. Numerical results 
of the new method are presented.
"""

reference_summary = """The paper explores the periodicity of approximately 155 days in sunspot activity during 1923–1933, using various data and a new diagnostic method for echo effects in power spectra."""


# Load model and tokenizer
model = LongT5ForConditionalGeneration.from_pretrained("./longt5_best_model")
tokenizer = AutoTokenizer.from_pretrained("./longt5_best_model")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Preprocess and generate
input_text = "summarize: " + text_to_summarize
inputs = tokenizer(input_text, return_tensors="pt", max_length=4096, truncation=True).to(device)

summary_ids = model.generate(
    inputs["input_ids"],
    max_length=256,
    min_length=30,
    length_penalty=2.0,
    repetition_penalty=1.2,
    num_beams=4,
    early_stopping=True
)

generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("\nGenerated Summary:\n", generated_summary)

# Evaluate with ROUGE
if reference_summary:
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    score = scorer.score(reference_summary, generated_summary)

    print("\nROUGE Scores:")
    for k, v in score.items():
        print(f"{k}: {v.fmeasure:.4f}")



Generated Summary:
 sunspot data from 1923 - 1933 (cycle 16) are analysed. a new method of the diagnosis of an echo-effect in the power spectrum is presented. numerical results of the new method are presented.

ROUGE Scores:
rouge1: 0.4516
rouge2: 0.1333
rougeL: 0.3226


In [6]:
! pip list

Package                      Version
---------------------------- ------------
absl-py                      2.3.1
accelerate                   0.31.0
aiohappyeyeballs             2.6.1
aiohttp                      3.12.15
aiosignal                    1.4.0
asttokens                    3.0.0
astunparse                   1.6.3
async-timeout                5.0.1
attrs                        25.3.0
certifi                      2025.7.14
charset-normalizer           3.4.2
click                        8.2.1
colorama                     0.4.6
comm                         0.2.2
datasets                     2.18.0
debugpy                      1.8.14
decorator                    5.2.1
dill                         0.3.8
evaluate                     0.4.1
exceptiongroup               1.3.0
executing                    2.2.0
filelock                     3.18.0
flatbuffers                  25.2.10
frozenlist                   1.7.0
fsspec                       2024.2.0
gast                         0